In [3]:
import xml.etree.ElementTree as ET
import mysql.connector
import logging
import uuid


In [133]:
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="AmeSame420",
        database="mydb"
    )


ns = {'ns': 'http://www.battlescribe.net/schema/catalogueSchema'}


def insert_faction(root, cursor):
    _name = root.get('name')
    _description = root.get('description')
    if _description is None:
        _description = ""
    _id = root.get('id')

    sql = "INSERT INTO Faction (name, description, id) VALUES (%s, %s,%s)"
    try:
        cursor.execute(sql, (_name, _description, _id))
    except Exception as e:
        logging.error("Error at %s", "insert_faction", exc_info=e)
        print(e)

    return _id


def insert_unit2(cursor, root, faction_id):
    sharedSelectionEntries = root.find('ns:sharedSelectionEntries', ns)
    for selectionEntry in sharedSelectionEntries:
        if "[Legends]" not in selectionEntry.get("name"):
            if selectionEntry.get("type") == "model" or selectionEntry.get("type") == "unit":

                _name = selectionEntry.get("name")
                _description = selectionEntry.get("description")
                _description = ""
                _id = selectionEntry.get("id")

                sql = "INSERT INTO Unit (name, description, id, faction_id) VALUES (%s, %s,%s,%s)"
                try:
                    cursor.execute(sql, (_name, _description, _id, faction_id))
                    parse_selectionEntry(cursor, selectionEntry, _id)
                    insert_keywords(cursor, selectionEntry, _id)
                except Exception as e:
                    logging.error("Error at %s", "insert_unit", exc_info=e)
                    print(e)

def insert_keywords(cursor, selectionEntry, unit_id):
    keywords = selectionEntry.findall(".//ns:categoryLink", ns)
    for keyword in keywords:
        _name = keyword.get("name")
        _id = keyword.get("targetId")
        sql = "INSERT IGNORE INTO ModelKeyWord (text, id) VALUES (%s, %s)"
        sql_map = "INSERT IGNORE INTO unit_has_ModelKeyword (Unit_id, ModelKeyword_id) VALUES (%s, %s)"
        try:
            cursor.execute(sql, (_name, _id))
            cursor.execute(sql_map, (unit_id, _id))
        except Exception as e:
            logging.error("Error at %s", "insert_keywords", exc_info=e)
            print(e)

def parse_selectionEntry(cursor, selectionEntry, unit_id):
    profiles = selectionEntry.findall(".//ns:profile", ns)

    for profile in profiles:
        if profile.get("typeName") == "Unit" or profile.get("typeName") == "Model":
            insert_model(cursor, profile, unit_id)
        if profile.get("typeName") == "Abilities":
            insert_abilities(cursor, profile, unit_id)
        if profile.get("typeName") == "Ranged Weapons" or profile.get("typeName") == "Melee Weapons":
            insert_weapons(cursor, profile, unit_id)


def insert_weapons(cursor, profile, unit_id):
    _name = profile.get("name")
    _id = profile.get("id")
    c = profile.find(".//ns:characteristics", ns)

    _range = None
    _strength = None
    _ap = None
    _damage = None
    _attack = None
    _hit = None


    for characteristic in c:
        if characteristic.get("name") == "Range":
            _range = characteristic.text
        if characteristic.get("name") == "S":
            _strength = characteristic.text
        if characteristic.get("name") == "AP":
            _ap = characteristic.text
        if characteristic.get("name") == "D":
            _damage = characteristic.text
        if characteristic.get("name") == "A":
            _attack = characteristic.text
        if characteristic.get("name") == "WS":
            _hit = characteristic.text
        if characteristic.get("name") == "BS":
            _hit = characteristic.text
        if characteristic.get("name") == "Keywords":
            print("insert_weapon_keywords")
            insert_weapon_keywords(cursor, characteristic, _id)
    sql = "INSERT INTO weapon (name, weapon_range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES (%s, %s,%s,%s,%s,%s,%s,%s,%s)"

    try:
        cursor.execute(sql, (_name, _range, _hit, _strength, _ap, _damage, _id, _attack, unit_id))
    except Exception as e:
        logging.error("Error at %s", "insert_weapons", exc_info=e)
        print("error!!!!")
        print(e)

def insert_weapon_keywords(cursor, characteristic, weapon_id):
    _keywords = characteristic.text
    # Check if the weapon has keywords
    if len(_keywords) < 3:
        return
    keywords = characteristic.text.split(", ")


    for keyword in keywords:
        keyword = keyword.replace("/n", "")
        keyword = keyword.replace(" ", "")
        if keyword == "":
            continue

        sql = "select * from weaponkeyword where name = (n) values (%s)"
        print("check1")
        result = cursor.execute(sql, (keyword))
        print("check2")
        if len(result) == 0:
            sql = "INSERT INTO weaponkeyword (name) VALUES (%s)"
            print("keyword", keyword)
            cursor.execute(sql, keyword)
            cursor.execute("SELECT id FROM weaponkeyword WHERE name = %s", keyword)
            result = cursor.fetchone()

            print("result", keyword)
            print(result)
            print(type(result))

        sql = "INSERT IGNORE INTO WeaponKeyWord (text, id) VALUES (%s, %s)"




def insert_abilities(cursor, profile, unit_id):
    _name = profile.get("name")
    _id = profile.get("id")
    _description = profile.findall(".//ns:characteristic", ns)[0].text

    sql = "INSERT INTO modelability (name, description, id) VALUES (%s, %s,%s)"
    sql_map = "INSERT INTO unit_has_modelability (unit_id, ModelAbility_id) VALUES (%s, %s)"
    try:
        cursor.execute(sql, (_name, _description, _id))
        cursor.execute(sql_map, (unit_id, _id))
    except Exception as e:
        logging.error("Error at %s", "insert_abilities", exc_info=e)
        print(e)


def insert_model(cursor, model, unit_id):
    _name = model.get("name")
    _id = model.get("id")

    move = None
    leadership = None
    save = None
    toughness = None
    wounds = None
    oc = None
    keywords = []
    for characteristics in model.findall(".//ns:characteristic", ns):

        if characteristics.get("name") == "M":
            move = characteristics.text
        elif characteristics.get("name") == "LD":
            leadership = characteristics.text
        elif characteristics.get("name") == "SV":
            save = characteristics.text
        elif characteristics.get("name") == "T":
            toughness = characteristics.text
        elif characteristics.get("name") == "W":
            wounds = characteristics.text
        elif characteristics.get("name") == "OC":
            oc = characteristics.text


        sql_models = "INSERT INTO Model (id, name, move, toughness,wounds, leadership, save, unit_id, oc) VALUES (%s, %s,%s,%s, %s, %s,%s,%s,%s)"


    try:
        cursor.execute(sql_models, (_id, _name, move, toughness, wounds, leadership, save, unit_id, oc))

    except Exception as e:
        logging.error("Error at %s", "insert_model", exc_info=e)
        print(e)


def parse_cat_file(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    return root


def ingest_data(root, cursor):
    faction_id = insert_faction(root, cursor)
    insert_unit2(cursor, root, faction_id)
    db_connection.commit()


def clear_all_tables(cursor):
    # Get all table names

    # Iterate over all tables and delete all records
    try:
        cursor.execute(f"DELETE FROM `CanTakeAsAllied`")
        cursor.execute(f"DELETE FROM `WeaponHasKeyword`")
        cursor.execute(f"DELETE FROM `WeaponKeyword`")
        cursor.execute(f"DELETE FROM `Unit_has_ModelKeyword`")
        cursor.execute(f"DELETE FROM `Weapon`")
        cursor.execute(f"DELETE FROM `Model`")
        cursor.execute(f"DELETE FROM `ModelKeyword`")
        cursor.execute(f"DELETE FROM `Unit_has_ModelAbility`")
        cursor.execute(f"DELETE FROM `ModelAbility`")
        cursor.execute(f"DELETE FROM `UnitCost`")
        cursor.execute(f"DELETE FROM `SpecialRules`")
        cursor.execute(f"DELETE FROM `Unit`")
        cursor.execute(f"DELETE FROM `Faction`")

        db_connection.commit()




    except mysql.connector.Error as err:
        print(f"Something went wrong: {err}")



In [134]:
db_connection = connect_to_db()
cursor = db_connection.cursor()


In [135]:
root = parse_cat_file("./wh40k-10e/Chaos - Death Guard.cat")
clear_all_tables(cursor)
ingest_data(root, cursor)

ERROR:root:Error at insert_unit
Traceback (most recent call last):
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_22716\2071436154.py", line 44, in insert_unit2
    parse_selectionEntry(cursor, selectionEntry, _id)
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_22716\2071436154.py", line 73, in parse_selectionEntry
    insert_weapons(cursor, profile, unit_id)
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_22716\2071436154.py", line 106, in insert_weapons
    insert_weapon_keywords(cursor, characteristic, _id)
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_22716\2071436154.py", line 132, in insert_weapon_keywords
    result = cursor.execute(sql, (keyword))
  File "C:\Users\krist\AppData\Local\Programs\Python\Python310\lib\site-packages\mysql\connector\cursor_cext.py", line 359, in execute
    prepared = self._cnx.prepare_for_mysql(params)
  File "C:\Users\krist\AppData\Local\Programs\Python\Python310\lib\site-packages\mysql\connector\connection_cext.py", line 880, in

insert_weapon_keywords
check1
Could not process parameters: str(LethalHits), it must be of type list, tuple or dict
insert_weapon_keywords
check1
Could not process parameters: str(LethalHits), it must be of type list, tuple or dict
insert_weapon_keywords
check1
Could not process parameters: str(Anti-INFANTRY4+), it must be of type list, tuple or dict
insert_weapon_keywords
check1
Could not process parameters: str(Anti-INFANTRY2+), it must be of type list, tuple or dict
insert_weapon_keywords
check1
Could not process parameters: str(LethalHits), it must be of type list, tuple or dict
insert_weapon_keywords
check1
Could not process parameters: str(Psychic), it must be of type list, tuple or dict
insert_weapon_keywords
check1
Could not process parameters: str(LethalHits), it must be of type list, tuple or dict
insert_weapon_keywords
check1
Could not process parameters: str(Melta2), it must be of type list, tuple or dict
insert_weapon_keywords
check1
Could not process parameters: str(Ignor